In [1]:
import os
HOME = os.getcwd()
print(HOME)

/home/manidhar/Desktop/AryanWork/yolov8-seg-training


In [ ]:
    from ultralytics import YOLO
    

    MODEL = "Trained Models/segmentation_trained_model.pt"
    model = YOLO(MODEL) 

In [2]:
def get_file_name_of_image_without_extension(image_path):
    import os
    image_filename_without_ext = os.path.splitext(os.path.basename(image_path))[0]
    return image_filename_without_ext

In [3]:

def get_predicted_mask_with_specific_classes(predicted_masks,predicted_classes,defined_classes):

    side_length = 640
    height = side_length
    width = side_length
    classes_available = False

    import numpy
    mask = numpy.zeros((height,width))

    for i in range(len(predicted_classes)):
        is_in_defined_classes = False

        #checking is annoatation is present in list of defined classes
        for annotation_class in defined_classes:
            if(annotation_class == predicted_classes[i]):
                is_in_defined_classes = True
                classes_available = True
        
        if(is_in_defined_classes):
            for row_index in range(len(predicted_masks[i])):
                for col_index in range(len(predicted_masks[i][row_index])):
                    if predicted_masks[i][row_index][col_index] == 1:
                        mask[row_index][col_index] = 1


    return mask,classes_available


In [4]:

def get_predicted_mask_from_model(filepath_of_image,defined_classes):

    result = model(filepath_of_image)

    predicted_masks = result[0].masks.masks.cpu().numpy()
    predicted_classes = result[0].boxes.cls.cpu().numpy()

    # print(predicted_masks)
    # print(predicted_class)

    return get_predicted_mask_with_specific_classes(predicted_masks,predicted_classes,defined_classes)



In [5]:
def get_actual_mask_from_polygon(file_name_of_label,defined_classes):

    import numpy
    from PIL import Image, ImageDraw

    side_length = 640 
    width = side_length
    height = side_length

    img = Image.new('L', (width, height), 0)

    file = open(file_name_of_label, mode = 'r', encoding = 'utf-8-sig')
    lines = file.readlines()
    file.close()
    
    classes_available = False

    for line in lines:
        line = line.split(' ')
        line = [i.strip() for i in line]
        class_of_annotation=int(line[0])
        is_in_defined_classes = False

        #checking is annoatation is present in list of defined classes
        for annotation_class in defined_classes:
            if(annotation_class == class_of_annotation):
                is_in_defined_classes = True
                classes_available = True

        if(is_in_defined_classes):
            polygon = []
            for i in range(1,len(line)):
                line[i]=float(line[i])*side_length
                polygon.append(line[i])
            ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)

    mask = numpy.array(img)
    print(mask)

    return mask,classes_available

In [6]:
def calculateIoU(gtMask, predMask):
    # Calculate the true positives,
    # false positives, and false negatives
    tp = 0
    fp = 0
    fn = 0

    for i in range(len(gtMask)):
        for j in range(len(gtMask[0])):
            if gtMask[i][j] == 1 and predMask[i][j] == 1:
                tp += 1
            elif gtMask[i][j] == 0 and predMask[i][j] == 1:
                fp += 1
            elif gtMask[i][j] == 1 and predMask[i][j] == 0:
                fn += 1

    # Calculate IoU
    iou = tp / (tp + fp + fn)

    return iou

In [7]:
def get_filepath_of_image_and_label(image_path): # update test folder location
    path_to_test_folder = "FINALDATASET/test/"
    filename_without_extension = get_file_name_of_image_without_extension(image_path)
    filename_of_image = HOME + "/" + path_to_test_folder + "images/" + filename_without_extension + ".jpg"
    filename_of_label = HOME + "/" + path_to_test_folder + "labels/" + filename_without_extension + ".txt"
    return filename_of_image,filename_of_label

In [8]:
def test_image_with_mask(mask,image_path):
    
    import cv2
    img = cv2.imread(image_path) # 640 x 640 x 3
    
    for i in range(640):
        for j in range(640):
            if mask[i][j]==0:
                img[i][j]=[0,0,0]
                
    cv2.imshow("img",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
def calculateIoUforimage(image_path,defined_classes):
    image_path,label_path = get_filepath_of_image_and_label(image_path)

    actual_mask,actual_mask_available = get_actual_mask_from_polygon(label_path,defined_classes)
    predicted_mask,predicted_mask_available = get_predicted_mask_from_model(image_path,defined_classes)

    if actual_mask_available or predicted_mask_available:
        iou = calculateIoU(actual_mask,predicted_mask)
        return iou,1
    return 0,0

In [ ]:
import glob

defined_classes = [6]

total_iou = 0
count = 0

for image_path in glob.glob(HOME +'/FINALDATASET/test/images/*.jpg')[:]:
  iou,cnt = calculateIoUforimage(image_path,defined_classes)
  total_iou = total_iou + iou
  count = count + cnt

average_iou = total_iou/count
print(average_iou)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


...

YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs


Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# defined_classes = [6]

# image_path = "Test_Images/2023-06-03-01_21_32-177914__00000008_jpg.rf.1a433f90789374c2587975eaec0b4a97.jpg"
# image_path,label_path = get_filepath_of_image_and_label(image_path)

# actual_mask,actual_mask_available = get_actual_mask_from_polygon(label_path,defined_classes)
# predicted_mask,predicted_mask_available = get_predicted_mask_from_model(image_path,defined_classes)

# # test_image_with_mask(actual_mask,image_path)

# if actual_mask_available or predicted_mask_available:
#     iou = calculateIoU(actual_mask,predicted_mask)
#     print("iou for the given image is " + str(iou))
# else:
#     print("Given classes not available in your test image!")


Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)


YOLOv8s-seg summary (fused): 195 layers, 11782696 parameters, 0 gradients, 42.5 GFLOPs


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
iou for the given image is 0.9130071099958177
